In [33]:
import os
import glob
from uuid import uuid4
from dotenv import load_dotenv
from sentence_transformers import SentenceTransformer

from pinecone import Pinecone
from pinecone import ServerlessSpec

from langchain import hub
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

from pymilvus import connections, Collection, FieldSchema, CollectionSchema, DataType

load_dotenv()

True

In [3]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['PINECONE_API_KEY']=os.getenv("PINECONE_API_KEY")

In [8]:
# Step 1: Load all PDFs from directory
pdf_dir = "./data/"
pdf_files = glob.glob(os.path.join(pdf_dir, "*.pdf"))

In [10]:
all_docs = []
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    all_docs.extend(loader.load())

In [13]:
print(f"Loaded {len(all_docs)} page documents (should cover >200 pages).")

Loaded 42 page documents (should cover >200 pages).


In [ ]:
# Step 2: Semantic Chunking
embedder = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
chunker = SemanticChunker(embedder, breakpoint_threshold_type="standard_deviation", min_chunk_size=200)
semantic_chunks = chunker.create_documents([doc.page_content for doc in all_docs])

print(f"Total semantic chunks: {len(semantic_chunks)}")

Total semantic chunks: 56


In [28]:
semantic_chunks[0].page_content

'arXiv:2405.10842v2  [cs.CV]  29 May 2024\nIEEE REVIEWS IN BIOMEDICAL ENGINEERING, VOL. XX, NO. XX, XXX X 2023 1\nAutomated Radiology Report Generation:\nA Review of Recent Advances\nPhillip Sloan, Philip Clatworthy , Edwin Simpson, and Majid Mirmehdi\nAbstract — Increasing demands on medical imaging de-\npartments are taking a toll on the radiologist’s ability to\ndeliver timely and accurate reports. Recent technological\nadvances in artiﬁcial intelligence have demonstrated grea t\npotential for automatic radiology report generation (ARRG ),\nsparking an explosion of research. This survey paper con-\nducts a methodological review of contemporary ARRG\napproaches by way of (i) assessing datasets based on\ncharacteristics, such as availability , size, and adoption rate,\n(ii) examining deep learning training methods, such as con-\ntrastive learning and reinforcement learning, (iii) explo ring\nstate-of-the-art model architectures, including variati ons\nof CNN and transformer models, (i

In [ ]:
texts = [chunk.page_content for chunk in semantic_chunks]
embeddings = embedder.embed_documents(texts)

In [31]:
len(embeddings)

56

In [35]:
# Connect to Milvus
connections.connect("default", host="localhost", port="19530")

MilvusException: <MilvusException: (code=2, message=Fail connecting to server on localhost:19530, illegal connection params or server unavailable)>